First things first.  Load the package.

In [1]:
Sys.setenv(R_CONFIG_ACTIVE = "jupyter")
suppressMessages({
  devtools::load_all()
  data(pcd)
})

metayer provides two methods for easing metaprogramming tasks, `wrapped_factory` and `with_monkey_patch`.

## wrapped_factory

It's easier to start with `wrapped_factory` which will allow us to wrap existing functions. One feature of `wrapped_factory` is that it replicates the function signature, including default values and items that would otherwise be lazily evaluated.  We note that wrappers should adopt the form `function(cmd, args, ...)`.  `wrapped_factory` will substitute `cmd` and `args` in the wrapped function.  `...` may be replaced with key-value pairs, and these will be injected into the wrapped function scope.

An example should be illustrative.  Let's define a wrapper that adds printf debugging to an existing function.  

In [12]:
# a simple "debug" wrapper
debug_wrapper <- function(cmd, args, label = NULL) { 
  
  # emit debugging information
  sprintf(">>> called '%s'\n", label) %>%
    cat(file = stdout())

  # call the original function
  do.call(cmd, args)
}

We'll apply this wrapper to a simple function:

In [13]:
# a very simple function
sum <- function(x, y) x + y

Now, invoke the `wrapped_factory` machinery to produce a wrapped function.  This will print the debugging information on stdout and the return the result.

In [14]:
dbg_sum <- wrapped_factory("sum", debug_wrapper, label = "..sum..")
dbg_sum(1, 2)

>>> called '..sum..'


[1] 3

It's instructive to inspect the structure of `dbg_sum`.  It has the same function signature as the original function; the body of the function is `debug_wrapper` with adapted `cmd` and `args`.  Specifically, `cmd` has been replaced with `sum`, and `args` has been replaced with a symbol-mapped list.  `wrapped_factory` makes these changes with the `substitute` function, and this will happen wherever these symbols are found.

In [16]:
dbg_sum

function (x, y) 
{
    sprintf(">>> called '%s'\n", label) %>% cat(file = stdout())
    do.call(sum, list(x = x, y = y))
}
<environment: 0x57a85ade80f0>

## with_monkey_patch

`with_monkey_patch` does roughly the same thing as `wrapped_factory`, but it is applied to functions defined in a namespace and only temporarily.  Changes will be restored on exit.

The following is a simple example.  Consider the behavior of "base::Sys.time": it captures the local time zone in its output.

In [46]:
# e.g., "PDT" in Seattle.
Sys.time()

[1] "2024-09-25 09:27:32 PDT"

That means that a function, like the one below, is inherently broken.  It's an inflexible implementation, only meaningful if the timezone is fixed apriori.

In [47]:
# hmm...
get_time_string <- function() {
  Sys.time() %>% as.character()
}
get_time_string()

[1] "2024-09-25 09:27:38.851042"

`with_monkey_patch` allows us to to fix `get_time_string` indirectly.  It does this by temporarily modifying the behavior of `base::Sys.time` in a scoped block of client code.  Here, calls to `base::Sys.time` will return UTC.

In [48]:
# monkey patch that forces base
with_monkey_patch(
  "base::Sys.time",
  # 
  wrapper = function(cmd, args, func) {    
    t <- do.call(func, args)
    .POSIXct(t, "UTC")
  },
  {
    # scoped block: calls to Sys.time will return UTC
    get_time_string()
  }
)

[1] "2024-09-25 16:27:42.723768"

In [49]:
# check the original behavior
get_time_string()

[1] "2024-09-25 09:27:42.908384"

### a real example

While the above would hopefully never happen in production code, there are examples where it's useful to modify an object created deep in a call stack.  

metayer found this machinery useful for affixing knitr hooks to rmarkdown documents.  Specifically, `pkgdown::build_article` invokes `rmarkdown::html_document`, but does so with limited configurability.  Our scenario requires a knitr hook, and while it is easy to modify an html_document object to install such a hook, the modification machinery wasn't available through the existing pkgdown API.  One proposal was to monkey path the behavior of `rmarkdown::html_document` so that it returned an object with the modifications that were necessary.

Ultimately, the monkey patching approach for `rmarkdown::html_document` was refactored away, but, for a time, it provide a viable solution to the original problem.